In [1]:
import torch
from transformers import AutoTokenizer
import datasets
from utils_generator import GenerationModel

from torch.utils.data import DataLoader, random_split
torch.manual_seed(9)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-06 16:52:28.572537: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 16:52:29.650591: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-06 16:52:29.650732: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libn

In [2]:
args = {
    "batch_size":16,
    "num_workers":0,
    "shuffle":True,
    "drop_last":False
    }

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")

model = GenerationModel("Helsinki-NLP/opus-mt-de-en")

/home/funny_shannon/.local/lib/python3.8/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


cuda


In [4]:
data_de = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','deu-eng',split=datasets.Split.TEST, 
                                verification_mode= datasets.VerificationMode.NO_CHECKS)
data_de_train, data_de_test = random_split(data_de, [0.7, 0.3])

data_fr = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','eng-fra', split=datasets.Split.TEST, 
                                verification_mode= datasets.VerificationMode.NO_CHECKS)\
                                    .rename_columns({'sourceLang': 'targetlang', 'targetlang': 'sourceLang', 
                                                     'sourceString': 'targetString', 'targetString': 'sourceString'})
data_fr_train, data_fr_test = random_split(data_fr, [0.7, 0.3])

# data_nld = datasets.load_dataset('Helsinki-NLP/tatoeba_mt','eng-nld',split=datasets.Split.TEST,
#                                   verification_mode= datasets.VerificationMode.NO_CHECKS)\
#                                     .rename_columns({'sourceLang': 'targetlang', 'targetlang': 'sourceLang', 
#                                                      'sourceString': 'targetString', 'targetString': 'sourceString'})

Found cached dataset tatoeba_mt (/home/funny_shannon/.cache/huggingface/datasets/Helsinki-NLP___tatoeba_mt/deu-eng/0.0.0/01e819f3f64a772a2ca70949061d295d3a2dc99d05183fe4776a3be23f75f619)
Found cached dataset tatoeba_mt (/home/funny_shannon/.cache/huggingface/datasets/Helsinki-NLP___tatoeba_mt/eng-fra/0.0.0/01e819f3f64a772a2ca70949061d295d3a2dc99d05183fe4776a3be23f75f619)


In [5]:
data_de_test[0]

{'sourceLang': 'deu',
 'targetlang': 'eng',
 'sourceString': 'Schach ist schwer.',
 'targetString': 'Chess is difficult.'}

In [6]:
loader_de_train = DataLoader(data_de_train, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

loader_de_test = DataLoader(data_de_test, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

loader_fr_train = DataLoader(data_fr_train, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

loader_fr_test = DataLoader(data_fr_test, batch_size=args['batch_size'], num_workers=args['num_workers'], 
                       shuffle=args['shuffle'], drop_last=args['drop_last'])

# loader_nld = DataLoader(data_nld, batch_size=args['batch_size'], num_workers=args['num_workers'], 
#                        shuffle=args['shuffle'], drop_last=args['drop_last'])

In [7]:
x_de_train = tokenizer(next(iter(loader_de_train))["sourceString"], return_tensors="pt", padding=True)
x_de_test = tokenizer(next(iter(loader_de_test))["sourceString"], return_tensors="pt", padding=True)


x_fr_train = tokenizer(next(iter(loader_fr_train))["sourceString"], return_tensors="pt", padding=True)
# x_fr_test = tokenizer(next(iter(loader_fr_test))["sourceString"], return_tensors="pt", padding=True)

# x_nld = tokenizer(next(iter(loader_nld))["sourceString"], return_tensors="pt", padding=True)

In [8]:
model.generate_bag(loader_de_train)

Generate bag of distribution:   0%|          | 0/769 [00:00<?, ?it/s]/home/funny_shannon/.local/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 512 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generate bag of distribution: 100%|██████████| 769/769 [02:24<00:00,  5.32it/s]


tensor([[1.9912e-01, 1.9238e-06, 3.9704e-04,  ..., 1.7847e-06, 1.7250e-06,
         0.0000e+00],
        [1.8176e-01, 1.7974e-06, 1.5423e-03,  ..., 1.6767e-06, 1.6458e-06,
         0.0000e+00],
        [1.8808e-01, 3.1588e-06, 1.4716e-04,  ..., 2.8378e-06, 2.6739e-06,
         0.0000e+00],
        ...,
        [1.8343e-01, 2.2108e-06, 2.0578e-04,  ..., 2.0458e-06, 1.9161e-06,
         0.0000e+00],
        [2.4672e-01, 2.0948e-06, 3.7988e-04,  ..., 1.9547e-06, 1.9018e-06,
         0.0000e+00],
        [6.9219e-02, 1.4555e-06, 5.2810e-02,  ..., 1.4299e-06, 1.4094e-06,
         0.0000e+00]], device='cuda:0')

In [9]:
anomaly= model.anomaly_score(x_de_test, 'Renyi', "s1", 0.1, temperature=1)

Computing divergence: 100%|██████████| 192/192 [00:00<00:00, 406.41it/s] 


In [10]:
anomaly

tensor([0.1145, 0.0887, 0.1311, 0.1680, 0.0970, 0.0746, 0.0040, 0.0858, 0.0784,
        0.1458, 0.0548, 0.0979, 0.0440, 0.0867, 0.0639, 0.0766],
       device='cuda:0')

In [11]:
anomaly_fr= model.anomaly_score(x_fr_train, 'Renyi', "s1", 0.1, temperature=1)

Computing divergence: 100%|██████████| 192/192 [00:00<00:00, 435.17it/s] 


In [12]:
threshold = model.define_threshold(loader_fr_test, 0.95, 'Renyi', "s1", 0.1, temperature=1)

Computing anomaly score for each data: 100%|██████████| 238/238 [05:46<00:00,  1.46s/it]


In [13]:
threshold

tensor(0.0862, device='cuda:0')

In [14]:
model.classify(x_de_train, threshold, 'Renyi', "s1", 0.1, temperature=1)

Computing divergence: 100%|██████████| 192/192 [00:00<00:00, 435.22it/s] 


tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False], device='cuda:0')

In [15]:
model.classify(x_fr_train, threshold, 'Renyi', "s1", 0.1, temperature=1)

Computing divergence: 100%|██████████| 192/192 [00:00<00:00, 435.14it/s] 


tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True], device='cuda:0')

In [11]:
anomaly, _ = model.anomaly_score(x_de_train, 'Renyi', "s0", 0.1, temperature=1)

In [12]:
anomaly

tensor([0.2623, 0.2500, 0.2557, 0.2670, 0.2444, 0.2570, 0.2797, 0.3035, 0.2568,
        0.2556, 0.2404, 0.2580, 0.3005, 0.2573, 0.2630, 0.2653],
       device='cuda:0')

In [13]:
#out = model.model.generate(**x_de, return_dict_in_generate=True,output_scores=True, num_beams=1)

In [14]:
#out.scores[0].shape

In [15]:
#torch.stack(list(out.scores), dim=0).permute(1,0,2)

In [16]:
#model.softmax(torch.stack(list(out.scores), dim=0).permute(1,0,2))

In [17]:
# anomaly_score_de, sequences_de = model.anomaly_score(x_de, 'Renyi', 0.1, temperature=1)

# anomaly_score_fr, sequences_fr = model.anomaly_score(x_fr, 'Renyi', 0.1, temperature=1)

# anomaly_score_nld, sequences_nld = model.anomaly_score(x_nld, 'Renyi', 0.1, temperature=1)

In [18]:
# anomaly_score_de

In [19]:
# anomaly_score_fr

In [20]:
# anomaly_score_nld

In [21]:
# print(torch.mean(anomaly_score_de))
# print(torch.std(anomaly_score_de))


In [22]:
# print(torch.mean(anomaly_score_fr))
# print(torch.std(anomaly_score_fr))

In [23]:
# print(torch.mean(anomaly_score_nld))
# print(torch.std(anomaly_score_nld))


In [24]:
# tokenizer.batch_decode(generated_ids.sequences, skip_special_tokens=True)

In [25]:
# x = tokenizer(data_nld[0]["targetString"], return_tensors="pt")

# generated_ids = model.generate(**x)
# tokenizer.batch_decode(generated_ids, skip_special_tokens=True)